In [7]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:1122/v1"
import time

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


chat_response = client.chat.completions.create(
    model="quant_lora",
    messages=[
        {"role": "system", "content": "你是一个人工智能助手."},
        {"role": "user", "content": prompt_template},
    ]
)
print("Chat response:", chat_response)

Chat response: ChatCompletion(id='cmpl-3f2f368638ad497d91f9f6ce02ba9fbf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"city": "三亚", "date": "2020-04-16"}<|im_end|>\n', role='assistant', function_call=None, tool_calls=None), stop_reason=151643)], created=1715825507, model='quant_lora', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=24, prompt_tokens=90, total_tokens=114))


In [10]:
prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''
Q_list=['2020年4月16号三亚下雨么?','青岛3-15号天气预报','5月6号下雪么, 城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么, 大概是1月15号左右, 我们3个人一起'] * 10

prompts = [prompt_template%(Q,) for Q in Q_list]

from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:1122/v1"
import time

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

start = time.time()

for prompt in prompts:
    chat_response = client.chat.completions.create(
        model="quant_lora",
        messages=[
            {"role": "system", "content": "你是一个人工智能助手."},
            {"role": "user", "content": prompt},
        ]
    )
    print("Chat response:",  chat_response.choices[0].message.content)
print(time.time() - start)


Chat response: {"city": "三亚", "date": "2020-04-16"}<|im_end|>

Chat response: {"city": "青岛", "date": "03-15"}<|im_end|>

Chat response: {"city": "威海", "date": "05-06"}<|im_end|>

Chat response: {"city": "青岛", "date": "2023-12-30"}<|im_end|>

Chat response: {"city": "北京", "date": "2023-06-01"}<|im_end|>

Chat response: {"city": "上海", "date": "01-03"}<|im_end|>

Chat response: {"city": "上海", "date": "08-08"}<|im_end|>

Chat response: {"city": "东北", "date": "01-15"}<|im_end|>

Chat response: {"city": "三亚", "date": "2020-04-16"}<|im_end|>

Chat response: {"city": "青岛", "date": "03-15"}<|im_end|>

Chat response: {"city": "威海", "date": "05-06"}<|im_end|>

Chat response: {"city": "青岛", "date": "12-30"}<|im_end|>

Chat response: {"city": "北京", "date": "06-01"}<|im_end|>

Chat response: {"city": "上海", "date": "01-03"}<|im_end|>

Chat response: {"city": "上海", "date": "08-08"}<|im_end|>

Chat response: {"city": "东北", "date": "01-15"}<|im_end|>

Chat response: {"city": "三亚", "date": "2020-04-16"}<

In [16]:
# 7B-chat-Int4:      16s
# Merge_lora-GPTQ    15s
# Merge_lora-AWQ     19s (need float16)

# Merge_lora         25s 

'{"city": "东北", "date": "01-15"}<|im_end|>\n'

In [ ]:
from vllm import LLM, SamplingParams
import time


# Sample prompts.
prompts = [
    "告诉我有关北京的特产",
] * 5

# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=400, ignore_eos=True)

# Create an LLM.
llm = LLM(model='gptq_qwen/', tokenizer='gptq_qwen',  quantization='gptq',
         trust_remote_code=True)

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''

Q_list=['2020年4月16号三亚下雨么?','青岛3-15号天气预报','5月6号下雪么, 城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么, 大概是1月15号左右, 我们3个人一起']

prompts = [prompt_template%(Q,) for Q in Q_list]

start = time.time()
outputs = llm.generate(prompts, sampling_params)
print(time.time()-start)


In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] ='0, 1'

In [2]:
! echo $CUDA_VISIBLE_DEVICES

0, 1


In [4]:
from vllm import LLM, SamplingParams
import time



# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=400, ignore_eos=True)

model_path = '/data/liucd/langchain/Qwen/Qwen1___5-7B-Chat-GPTQ-Int4/'
# Create an LLM.
llm = LLM(model= model_path, 
           quantization='gptq',
          max_model_len=8192,  # 这个是必要的，否则
          # tensor_parallel_size=2, gpu_memory_utilization=0.8,
         trust_remote_code=True)

WARNING 05-16 17:26:01 config.py:169] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-16 17:26:01 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='/data/liucd/langchain/Qwen/Qwen1___5-7B-Chat-GPTQ-Int4/', speculative_config=None, tokenizer='/data/liucd/langchain/Qwen/Qwen1___5-7B-Chat-GPTQ-Int4/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-16 17:26:01 utils.py:608] Found nccl from library /home/liuchangdong/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-16 17:26:04 selector.py:77] Cannot use FlashAttention backend because the flash_attn package is not found. Please install it for better performance.
INFO 05-16 17:26:04 selector.py:33] Using XFormers backend.
INFO 05-16 17:26:13 model_runner.py:173] Loading model weights took 5.5124 GB
INFO 05-16 17:26:18 gpu_executor.py:119] # GPU blocks: 3578, # CPU blocks: 512
INFO 05-16 17:26:23 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-16 17:26:23 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memor

In [24]:
prompts = [
       "告诉我有关上海的特产有哪些？" 
] 


start = time.time()
outputs = llm.generate(prompts, sampling_params)
num_tokens = sum([len(output.outputs[0].token_ids) for output in outputs]) 
spend_time = time.time() - start
print(outputs[-1].outputs[0].text)
print(num_tokens / spend_time )


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.95s/it]

 有关上海的特产，这里罗列了一些具有代表性的：

1. **小笼包**：上海特色小吃，以鲜肉和皮薄馅多著称，尤其是南翔馒头店的最为著名。

2. **生煎**：上海人常吃的早点，底酥皮脆，里面是软嫩的猪肉和汤汁，一咬一兜油。

3. **蟹壳黄**：一种酥皮点心，类似月饼，以黄油和芝麻制成，上海人逢年过节常做。

4. **糟肉**：用猪五花肉腌制，再配以糟酒，口感醇厚，常作为节日菜肴。

5. **上海酒酿圆子**：糯米制成的甜品，内含酒糟，甜而不腻，冬季必备。

6. **杨浦糕点**：杨浦区有特色的糕点，如杏仁糕、芝麻糖等。

7. **南翔蟹粉小笼**：与小笼包并列的特色小吃，蟹粉口感丰富。

8. **糕团**：上海本地的糯米团，种类多样，如咸蛋黄、豆沙等口味。

9. **毛豆腐**：上海特色豆制品，口感细腻，制作工艺独特。

10. **鲜肉月饼**：中秋节特色，肉馅和莲蓉搭配，味道醇厚。

11. **双皮奶**：甜品，上海人喜欢的乳制品，口感细腻，奶香味浓。

12. **糖醋排骨**：上海家常菜，色泽红亮，酸甜可口。

13. **上海炒面**：将面条炒至微黄，加入蔬菜、肉类等，是上海人的快餐。

14. **阿婆豆腐**：地道上海小吃，豆腐鲜嫩，汤汁浓郁。

以上只是上海特色的一部分，实际上上海美食种类繁多
135.21338528387636


In [7]:
#  tp    1     2
# 1     71     
# 10    224
# 100   894
# 500   1825
# 1000  2022   2042
# 10000 2258

# 张量并行数量增大会明显降低显存占用，但是不会增大推理速度，甚至略微降低
#       chat  AWQ(1.5) GPTQ(1.5)  MyGPTQ    GPTQ(1)
# 1     71      100    134     135
# 10    224     324    360     473
# 100   894     910    980     827           827           
# 500   1825    1553   1526    858
# 1000  2022    1642   1828
# 10000 2258    1603   2159

# batch size 小于100时，推理速度受限于内存中传输权重的带宽，量化模型较小所以可以更快
# batch size 大时，推理速度受限于计算，量化模型的反量化操作增加了计算资源，推理速度更慢

1732

In [ ]:
outputs[0].outputs[0].text

In [26]:
str = '北京的特产有“北京烤鸭”、“豆汁焦圈”、“羊蝎子”和“冰糖葫芦”等，其中“北京烤鸭”是最具代表性的，它以其独特的烹饪技术和美味而闻名；“豆汁焦圈”是一种传统的北京小吃，豆汁是用绿豆发酵而成，焦圈是用面粉和糖制成；“羊蝎子”是一种独特的烹饪方法，它使用羊的脊髓，经过烹饪后变得鲜美可口；“冰糖葫芦”是一种以山楂、葡萄、苹果等水果串起来，外面包裹冰糖的特色小吃，口感酸甜可口。'

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    '/data/liucd/langchain/Qwen/Qwen1___5-7B-Chat-AWQ', # path to the output directory
    trust_remote_code=True
)

out = tokenizer(str)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
len(out['input_ids'])

134

In [31]:
out

{'input_ids': [68990, 9370, 112984, 18830, 2073, 68990, 102554, 105397, 854, 91956, 99955, 102514, 100479, 100044, 854, 91956, 101187, 116558, 44729, 854, 33108, 2073, 100038, 100443, 111065, 854, 49567, 3837, 90919, 2073, 68990, 102554, 105397, 854, 104890, 76813, 99661, 104196, 3837, 99652, 109859, 105071, 111611, 107772, 106800, 68536, 109836, 24968, 2073, 99955, 102514, 100479, 100044, 854, 101158, 105062, 68990, 107600, 3837, 99955, 102514, 20412, 11622, 118693, 108279, 106042, 3837, 100479, 100044, 20412, 11622, 112398, 33108, 100443, 108711, 24968, 2073, 101187, 116558, 44729, 854, 101158, 105071, 111611, 39907, 3837, 99652, 37029, 101187, 9370, 109257, 103661, 3837, 101897, 111611, 33447, 101197, 99705, 57566, 30440, 39426, 24968, 2073, 100038, 100443, 111065, 854, 101158, 23031, 57811, 121142, 5373, 101580, 5373, 104167, 49567, 104618, 51575, 99793, 3837, 102586, 108232, 100038, 100443, 9370, 100175, 107600, 3837, 107816, 99918, 100475, 30440, 39426, 1773], 'attention_mask': [

In [57]:
!curl http://localhost:1212/v1/completions \
-H "Content-Type: application/json" \
-d '{
"model": "facebook/opt-125m",
"prompt": "San Francisco is a",
"max_tokens": 7,
"temperature": 0
}'

SyntaxError: unmatched '}' (3344056460.py, line 6)